In [1]:
# working with normal beam 2, without minlength control

import json
import nltk

beam_nomulti = './scores/lang-att-beam2-30k_minlength.json'
beam_multi = './scores/lang+att-beam2-30k_minlength.json'
beam_nomulti_att = './scores/lang-att+beam2-30k_minlength.json'
beam_multi_att = './scores/lang+att+beam2-30k_minlength.json'

with open(beam_nomulti, 'r') as f:
    beam_nomulti = json.load(f)
with open(beam_multi, 'r') as k:
    beam_multi = json.load(k)
with open(beam_nomulti_att, 'r') as l:
    beam_nomulti_att = json.load(l)
with open(beam_multi_att, 'r') as p:
    beam_multi_att = json.load(p)
    
hypotheses = [beam_nomulti, beam_multi, beam_nomulti_att, beam_multi_att]

In [2]:
def extract_hyp(data_dict):
    hyps_str = ''
    for item in data_dict:
        hyps_str += item['hypotheses']
        hyps_str += ' '
    out = nltk.tokenize.word_tokenize(hyps_str)
    return out

In [3]:
# vocabulary size (number of types)

for each_set in hypotheses:
    this_set_hyps_words = extract_hyp(each_set)
    print(len(this_set_hyps_words))
    print(len(set(this_set_hyps_words)))
    print()

152986
400

151964
425

153051
400

152552
434



In [4]:
# number of nouns and unique nouns (nouns describe objects, so it is a rough indicator of object mention)

is_noun = lambda pos: pos[:2] == 'NN'
for each_set in hypotheses:
    this_set_hyps_words = extract_hyp(each_set)
    this_set_nouns = [word for (word, pos) in nltk.pos_tag(this_set_hyps_words) if is_noun(pos)]
    print(len(this_set_nouns))
    print(len(set(this_set_nouns)))
    print()

37949
295

38354
320

38489
296

38489
325



In [5]:
def repetetive_nouns(data_dict):
    all_ratios = 0
    for item in data_dict:
        hyp = item['hypotheses'].split('.')
        this_hyp_nouns = {}
        num_of_rep_nouns = 0
        for elem in hyp:
            out = nltk.tokenize.word_tokenize(elem)
            nouns = [word for (word, pos) in nltk.pos_tag(out) if is_noun(pos)]
            for n in nouns:
                if n not in this_hyp_nouns:
                    this_hyp_nouns[n] = 1
                else:
                    this_hyp_nouns[n] += 1
        for each_n, freq in this_hyp_nouns.items():
            if freq != 1:
                # if some noun repeats > 1 times
                num_of_rep_nouns += 1
        ratio = num_of_rep_nouns / len(this_hyp_nouns)
        all_ratios += ratio
    return all_ratios / len(data_dict)

In [6]:
# average ratio of repetetive nouns / all unique nouns per paragraph
# smaller means more unique nouns in a generated paragraph on average
# nouns repeated more than once / all nouns

# 'in a single paragraph on average, 49% of the nouns are occuring more than 1 time in its sentences'
# models with multimodal input demonstrate more diversity in terms of nouns they contain
# this number should be high enough for connectivity, but not too much
# IDEA: introduce n-gram penalty on the level of words and/or the level of sentences?
# this would make av_rep_ratio_noun 'better', and improve generation evaluation scores

for each_set in hypotheses:
    av_rep_ratio_noun = repetetive_nouns(each_set)
    print(av_rep_ratio_noun)

0.49193992360926664
0.4729834119498632
0.4914266450927754
0.4774229620653875


|   |                                 | # of words | # of types | # of nouns | # of unique nouns |
|---|---------------------------------|------------|------------|------------|-------------------|
|   | BASELINE MULTIMODAL- ATTENTION- | 115,753    | 413        | 28,337     | 308               |
|   | BASELINE MULTIMODAL+ ATTENTION- | 116,381    | 437        | 28,755     | 321               |
|   | BASELINE MULTIMODAL- ATTENTION+ | 118,298    | 416        | 29,094     | 317               |
|   | BASELINE MULTIMODAL+ ATTENTION+ | 113,042    | 441        | 27,914     | 327               |